# TRPO和PPO

## I. 从vanilla PG到Natural PG，再到TRPO和PPO的出发点和算法

### I.1 背景知识：RL和DL样本分布的差异

#### I.1.1 RL和DL样本分布的差异

- 一般的DL中样本是iid的，也就是说通常可以假设样本都是来自同一个分布。但是这个假设在RL中不成立。
- RL中有两个因素导致iid不成立：
  1. 如果策略稳定，并且状态s的分布达到了稳定的d(s)。按照episodes来收集数据集，每个episode(trajectory)上，连续的time-steps中，前一步的状态必然影响后一步的状态，所以每个step上收集到的数据是相关的。
  2. 在算法迭代过程中，策略在不断更新，所以状态s是non-stationary distribution，基本不会处于稳定的d(s)上。也就是说，收集到的数据并不是来自相同的分布。这里还有一个因素让策略的更新过程并不smooth，也就是一次迭代产生的两个策略之间差别可能很大：
     - 算法迭代的参数是$\pi(\theta)中的\theta$，<font color=orange>NN中可以控制步幅让DL中的参数迭代量很小，这只能让参数的变化幅度小，但是从结果来看，策略本身的变化却可能很大。</font>
- 上述RL中数据分布的相关性和不稳定性会让RL算法迭代过程中reward波动很大。常见的现象是，迭代过程中很长时间reward都很小，然后过了很久才突然有一个很大的提升。它不会像supervised DL中，指标会逐步稳定上升。

#### I.1.2 基于vanilla PG的AC算法的两大问题

  1. 很难选择stepsize：这个问题在RL中比在深度学习中带来的后果更糟糕。
     - 由于数据分布不稳定，很难像DL中那样找到一个大小合适的stepsize，并且在整个训练过程中几乎不用太调整这个stepsize。在DL中，可以用normalization的方法来稳定让网络中的变量分布，然后用更好的优化器（如Adam）来拟合二阶优化，达到控制参数不同维度的步幅效果。这样就能让迭代过程稳定。但如前文分析，即使这样做也只能稳定网络参数的变化，却无法稳定策略本身，所以需要有办法稳定策略本身的变化。
     - 而且错误的stepsize带来的后果也更糟糕。如果stepsize略大，policy gradient得到的策略可能是一个非常差的策略。和DL中不一样，此时新的数据就会是用这个糟糕的策略生成的，这会进入无法恢复的恶性循环。
  2. sample efficiency很低，同样因为是on-policy算法，一个batch的trajectory用一次就扔掉。

### I.2 PG类算法进化

#### 1. vanill PG

- <font color=blue>**出发点和想解决的问题**：</font>
  1. 处理策略是stochastic policy的场景。
  2. 同时也能更方便地处理连续-高维的action space。

- <font color=green>**solution**：</font>
  1. 先用$max_\theta J(\theta)$为目标，计算策略梯度$\nabla_\theta J(\theta)$
  2. 在迭代中，为了降低梯度在迭代过程中的variance，在梯度表达式中加上了rewards-to-go和baseline两处调整

- <font color=red>**问题**：</font>在解决实际问题时，还是经常出现难以收敛到local min的问题。

#### 2. Natural PG

- <font color=blue>**出发点和想解决的问题**：</font>解决vanilla PG使用时训练难，不容易收敛到local min的问题

- <font color=green>**solution**：</font>Natural PG的一系列paper分析了迭代过程variance大的原因，并提出了两类主要的解决思路。
  1. kakade 2002的文章为代表的思路：找到每一步都能保证target improvement的方法，在算法中通过approximate optimal policy来让converge的过程更顺利。
  2. Peters 2008的文章代表的思路：分析了除梯度variance之外，导致vanilla PG的优化过程收敛困难的原因，发现主要问题出在目标分布的ill-condition上。这会让step-size的控制非常困难。提出两种技巧来解决问题：
     1. 用带约束的优化目标来限制step-size，控制新旧策略之间的KL divergence差异。
     2. 用Fisher information matrix来近似求解有约束的优化问题。

- <font color=red>**问题**：</font>
  1. kakade提出的monotonic improvement方法仅适用于mixture policy，在实践中不具备一般性，因此实际使用不广泛。
  2. Peters使用了正确的目标，但是求解优化问题使用的方法计算很繁琐

#### 3. TRPO

- <font color=blue>**出发点和想解决的问题**：</font>
  - 在kakade的思路基础上，找到适用于一般化的stochastic policy的monotonic improvement条件。并以它为依据，找到实践性强的approximate optimal policy来让converge的过程更顺利。

- <font color=green>**solution**：</font>
  - 通过数学分析，找到了能够让stochastic policy实现monotonic improvement的优化目标。由于该目标对应的step-size过于保守，对其进行了一些approximation，得到了和Peters 2008文章中一样的以KL divergence为了约束的优化目标。
  - 还找到了比Peters 2008paper中更高效的求解该优化问题的方法：使用conjugate gradient。

#### 4. PPO

- <font color=blue>**出发点和想解决的问题**：</font>利用现有的深度学习框架来训练RL算法，同时保留TRPO的两个优点：1)data efficiency；2)approximate monotonic improvement feature

- <font color=green>**solution**：</font>
  - 通过1）将梯度转化为surrogate loss；2）使用只用一阶梯度就能迭代求解的loss形式，使得算法求解过程能够使用DL framework。
  - 提出了两种算法对应两种不同的优化目标。
    - 第一种使用一阶梯度求解TRPO解优化问题，将硬性约束的优化问题中的约束以penalty形式加入优化目标。
    - 第二种对TRPO中的优化目标进行改造，使用clipped surrogate object。

## II. TRPO

### II.1 构造具备approximate monotonic improvement性质的优化目标

- <font color=norange>**思路**：</font>
  1. <font color=norange>先分析stochastic policy条件下保证monotonic improvement的策略迭代应该满足什么条件
  2. <font color=norange>证明在deterministic policy的条件下policy iteration的策略迭代方式满足该条件，因此policy iteration在优化正确的RL目标。</font>
  3. <font color=norange>如果想要处理stochastic policy，第一步分析得到的‘正确’的优化目标形式很难直接做优化。因此对它做调整，用调整后的表达式作为policy gradient的目标
  4. <font color=norange>证明在一定的约束条件下，调整后的policy gradient的目标与‘正确’目标的差异可以控制在一个明确的bound之内，且bound的大小可以通过新旧策略之间的差异来控制。
  5. <font color=norange>由于bound的存在且可控，进一步证明，在调整后的目标表达式加上一定的条件约束后，只要能让新目标monotonic improvement，就能保证原目标的monotonic improvement
  6. <font color=norange>上一步中得到的monotonic improvement算法目标要求的step-size非常保守，实践中会让迭代过程进展缓慢，因此放松条件，折衷取一个优化效率更高的approximate monotonic improvement算法.

#### II.1.1 每次迭代能做到monotonic improvement的条件

  - 符号：<font color=brown>$\bar\theta$表示new policy参数，$\theta$表示old policy参数</font>

- **一次迭代前后的差异可以表示为**：
$$\begin{align}
J(\bar \theta)-J(\theta) & = J(\bar \theta)- E_{s_0\sim P(s_0)}[V^{\pi_{\theta }}(s_0)]，这里P(s_0)不取决于\theta \\
& = J(\bar \theta)- E_{\tau\sim P_{\bar\theta}(\tau)}[V^{\pi_{\theta }}(s_0)]，\tau的分布不改变s_0的边际分布，所以\tau可以改取任意分布，只要它的初始状态满足P(s_0)就行\\
& = J(\bar \theta)- E_{\tau\sim P_{\bar\theta}(\tau)}\left[\sum_{t=0}^{\infty}\gamma^tV^{\pi_{\theta }}(s_t)-\sum_{t=1}^{\infty}\gamma^tV^{\pi_{\theta }}(s_t)\right]\\
& = J(\bar \theta)- E_{\tau\sim P_{\bar\theta}(\tau)}\left[\sum_{t=0}^{\infty}\left(\gamma^tV^{\pi_{\theta }}(s_t)-\gamma^{t+1}V^{\pi_{\theta }}(s_{t+1})\right)\right]\\
& = J(\bar \theta)+ E_{\tau\sim P_{\bar\theta}(\tau)}\left[\sum_{t=0}^{\infty}\gamma^t\left(\gamma V^{\pi_{\theta }}(s_{t+1})-V^{\pi_{\theta }}(s_t)\right)\right]\\
&= E_{\tau\sim P_{\bar\theta}(\tau)}\left[\sum_{t=0}^{\infty }\gamma^tr(s_t,a_t)\right]+ E_{\tau\sim P_{\bar\theta}(\tau)}\left[\sum_{t=0}^{\infty}\gamma^t\left(\gamma V^{\pi_{\theta }}(s_{t+1})-V^{\pi_{\theta }}(s_t)\right)\right]\\
&= E_{\tau\sim P_{\bar\theta}(\tau)}\left[\sum_{t=0}^{\infty }\gamma^t\left(r(s_t,a_t)+\gamma V^{\pi_{\theta }}(s_{t+1})-V^{\pi_{\theta }}(s_t)\right)\right]\\
& = E_{\tau\sim P_{\bar\theta}(\tau)}\left[\sum_{t=0}^{\infty }\gamma^tA^{\pi_{\theta}}(s_t,a_t) \right]
\end{align}$$

- <font color=red>说明：第二个等式成立是因为$P(s_0)$是环境决定的，不受策略影响。因此trajectory可以取任意分布，只要满足该分布下$s_0$的边际分布是$P(s_0)$就行。</font>

- **上式右边RHS部分可以分解表示为**: 
$$\begin{align}
J(\bar \theta)-J(\theta)& =\underset{\tau\sim P_{\bar \theta}(\tau)}{E}\left[\sum_t\gamma^tA^{\pi_{\theta}}(s_t,a_t) \right]\\
&= E_{s_t\sim P_{\bar \theta }(s_t)}E_{a_t\sim \pi_{\bar \theta }(a_t|s_t)}\sum_t\gamma ^tA^{\pi_{\theta }}(s_t,a_t)\\
&= \sum_{s\in\mathcal{S}}P_{\bar \theta }(s_t=s)\sum_{a\in\mathcal{A}}\pi_{\bar \theta }(a|s)\sum_t\gamma ^tA^{\pi_{\theta }}(s_t,a_t)\\
&= \sum_{s\in\mathcal{S}}\sum_t\gamma ^tP_{\bar \theta }(s_t=s)\sum_{a\in\mathcal{A}}\pi_{\bar \theta }(a|s)A^{\pi_{\theta }}(s_t,a_t)\\
&= \sum_{s\in\mathcal{S}}\rho_{\bar \theta }(s)\sum_{a\in\mathcal{A}}\pi_{\bar \theta }(a|s)A^{\pi_{\theta }}(s_t,a_t)\\
其中，\rho_{\bar \theta }(s) & =\sum_t\gamma ^tP_{\bar \theta }(s_t=s)，是unnormalized\ discounted\ visitation\ frequency.
\end{align}$$

- 从上式可见，<font color=green>**如果迭代要做到monotonic improvement，就要满足：对所有states，新策略都要有非负的advantage。**</font>
  - 从公式上看，如果不论s取什么值，都满足以下条件，除非advantage处处为0，那么迭代得到的新策略一定能improve target。即使advantage处处为0，至少新的策略不比旧策略更差。
$$\sum_{a\in\mathcal{A}}\pi_{\bar \theta }(a|s)A^{\pi_{\theta }}(s_t,a_t)\ge 0$$

#### II.1.2 deterministic policy的条件下<font color=orange>policy iteration</font>满足上述条件

- 在deterministic policy的条件下给定state s时，action是唯一确定的。在这类策略的policy iteration中，迭代条件是，所有state s条件的的action都满足：
$$\begin{align}
\pi_{\bar \theta}(s) 
& =\underset{a}{argmax} Q^{\pi_\theta}(s,a)\\
& =\underset{a}{argmax} [Q^{\pi_\theta}(s,a)-V^{\pi_\theta}(s)],\because V^{\pi_\theta}(s)不受a的影响\\
& =\underset{a}{argmax} A^{\pi_\theta}(s,a)\\
\end{align}$$

- 从该迭代条件可知，只要至少有一个state-action pair的advantage是正值，那么该state的visitation probability大于0，那么按照上述方式迭代得到的新策略必然满足monotonic improvement.

- <font color=blue>**结合policy iteration的执行过程来看，monotonic improvement的目标表达式本质上是soft version of policy iteration。**</font>
  - 因为policy iteration有两步：
     1. step1: policy evaluation是在求当前策略(旧策略)下的Advantage。也就是计算$A^{\pi_{\theta}}(s_t,a_t)$
     2. step2: policy improvement是用第一步得到的Advantage来更新得到新的策略
$${\color{red}{\bar \theta}} = \underset{{\color{red}{\bar \theta}}}{argmax}\ E_{\tau\sim P_{{\color{red}{\bar \theta}}}(\tau)}\left[\sum_{t=0}^{\infty }\gamma^t{\color{green}{A^{\pi_{\theta}}(s_t,a_t)}} \right]$$
  - 只不过policy iteration中的policy是deterministic的，此时action的概率调整在0和1之间跳转。如果将这种策略调整方式视为hard mode，那么policy gradient中的策略变化方式就是soft mode。因为此时策略是stochastic policy，action的概率取值空间是连续的，因此每一次迭代的更新时，策略的调整是在连续空间上渐进变化的。

#### II.1.3 <font color=orange>policy gradient</font>的优化目标和上述policy iteration的优化目标近似

- 如果将策略一般化为stochastic policy，直接用monotonic improvement的条件做为目标做优化很难。因为该目标中new policy的位置在期望分布里面($E_{\tau\sim P_{\color{red}{\bar\theta} }(\tau)}$)，这种形式的期望值既不能直接抽样，也不能直接求梯度。常见的处理方法是用importance sampling的方式将目标参数移出当前位置。

$$\begin{align}
E_{\tau\sim P_{\bar\theta}(\tau)}\left[\sum_{t=0}^{\infty }\gamma^tA^{\pi_{\theta}}(s_t,a_t) \right]
&= \sum_tE_{s_t\sim P_{\bar \theta }(s_t)}E_{a_t\sim \pi_{\bar \theta }(a_t|s_t)}[\gamma ^tA^{\pi_{\theta }}(s_t,a_t)]\\
&= \sum_tE_{s_t\sim P_{\bar \theta }(s_t)}E_{a_t\sim \pi_{\theta }(a_t|s_t)}[\frac{\pi_{\bar \theta }(a_t|s_t)}{\pi_{\theta }(a_t|s_t)} \gamma ^tA^{\pi_{\theta }}(s_t,a_t)], importance\ sampling\\
&= \sum_tE_{s_t\sim P_{\color{red}{\theta} }(s_t)}\frac{P_{\bar \theta}(s_t)}{P_{\theta}(s_t)}E_{a_t\sim \pi_{\theta }(a_t|s_t)}[\frac{\pi_{\bar \theta }(a_t|s_t)}{\pi_{\theta }(a_t|s_t)} \gamma ^tA^{\pi_{\theta }}(s_t,a_t)]\\
&\approx \sum_tE_{s_t\sim P_{\color{red}{\theta} }(s_t)}[E_{a_t\sim \pi_{\theta }(a_t|s_t)}[\frac{\pi_{\bar \theta }(a_t|s_t)}{\pi_{\theta }(a_t|s_t)} \gamma ^tA^{\pi_{\theta }}(s_t,a_t)]]\\
& = E_{\tau\sim P_{\color{red}{\theta} }(\tau)}\left[\sum_t\frac{\pi_{\bar \theta }(a_t|s_t)}{\pi_{\theta }(a_t|s_t)} \gamma ^tA^{\pi_{\theta }}(s_t,a_t)\right]
\end{align}$$

- $P_{\bar\theta}(s_t)$的信息很难获取，所以直接处理原目标在importance sampling之后仍然很难。当新旧策略差异不大的时候，可以认为$P_{\bar \theta}\approx P_{\theta}$，因此一种思路是用近似目标代替原目标：

$$E_{\tau\sim P_{\bar\theta}(\tau)}\left[\sum_{t=0}^{\infty }\gamma^tA^{\pi_{\theta}}(s_t,a_t) \right]
\approx E_{\tau\sim P_{\color{red}{\theta} }(\tau)}\left[\sum_t\frac{\pi_{\bar \theta }(a_t|s_t)}{\pi_{\theta }(a_t|s_t)} \gamma ^tA^{\pi_{\theta }}(s_t,a_t)\right]$$

- 可以证明，新旧策略差异不大时，近似目标与原目标之间的差异有明确的bound。

#### II.1.4 用policy gradient近似policy iteration的误差bound分析: 简化版

- 当策略是deterministic时的bound分析
  - <img src='pics/bound_deterministic.png' width='65%'>

- 将策略一般化为任意分布时的bound分析
  - <img src='pics/bound_general.png' width='65%'>

- 将上面结论代入目标表达式有：
  - <img src='pics/policy_gradient_approx.png' width='65%'>

- 所以，定性地来看，只要$\epsilon$的值足够小，那么：
$$\sum_tE_{s_t\sim P_{\color{green}{\bar \theta}}(s_t)}\left[E_{a_t\sim \pi_{\theta }(a_t|s_t)}[\frac{\pi_{\bar \theta }(a_t|s_t)}{\pi_{\theta }(a_t|s_t)} \gamma ^tA^{\pi_{\theta }}(s_t,a_t)]\right] 
\approx \sum_tE_{s_t\sim P_{\color{red}{\theta}}(s_t)}\left[E_{a_t\sim \pi_{\theta }(a_t|s_t)}[\frac{\pi_{\bar \theta }(a_t|s_t)}{\pi_{\theta }(a_t|s_t)} \gamma ^tA^{\pi_{\theta }}(s_t,a_t)]\right]\\
$$

#### II.1.5 误差bound分析：严谨版

- 用total variation divergence定义两种策略之间的距离，表示为：
$$\begin{align}
D_{TV}(p||q)& = \frac{1}{2}\sum_i|p_i-q_i| \\
进一步定义：\\
D_{TV}^{max}(\pi_{\bar\theta}||\pi_{\theta}) & = max_sD_{TV}(\pi_{\bar\theta}(·|s)||\pi_{\theta}(·|s))
\end{align}$$

- 定义符号：
$$\begin{align}
\bar A(origin) = \sum_tE_{s_t\sim P_{\color{green}{\bar \theta}}(s_t)}\left[E_{a_t\sim \pi_{\theta }(a_t|s_t)}[\frac{\pi_{\bar \theta }(a_t|s_t)}{\pi_{\theta }(a_t|s_t)} \gamma ^tA^{\pi_{\theta }}(s_t,a_t)]\right]\\ 
\bar A(approx) = \sum_tE_{s_t\sim P_{\color{red}{\theta}}(s_t)}\left[E_{a_t\sim \pi_{\theta }(a_t|s_t)}[\frac{\pi_{\bar \theta }(a_t|s_t)}{\pi_{\theta }(a_t|s_t)} \gamma ^tA^{\pi_{\theta }}(s_t,a_t)]\right]
\end{align}$$

- <font color=blue>**定理1**：</font>
$$\begin{align}
取\alpha =D^{max}_{TV}(\pi_{new}||\pi_{old}),& 有：\\
&\bar A(approx) - \bar A(origin) \le \frac{4\epsilon \gamma }{(1-\gamma )^2}\alpha ^2\\
&其中，\epsilon = \underset{s,a}{max}|A_{\pi_{old}}(s,a)|
\end{align}$$

- 根据Pinsker's inequality, total variational difference与KL divergence之间有下述关系：<font color=brown>[证明见附页，ref deepseek]</font>
$$|\pi_{\bar \theta}(a_t|s_t)-\pi_{\theta}(a_t|s_t)| \le \sqrt{0.5*D_{KL}(\pi_{\bar \theta}(a_t|s_t)||\pi_{\theta}(a_t|s_t))}$$
  - <font color=green>因此可以将前述约束条件转化为使用的KL divergence约束，得到下面等价定理。</font>

- <font color=blue>**定理2**：</font>
$$\begin{align}
&\bar A(approx) - \bar A(origin) \le \frac{4\epsilon \gamma }{(1-\gamma )^2}D^{max}_{KL}(\pi_{old}||\pi_{new})\\
&其中，\epsilon = \underset{s,a}{max}|A_{\pi_{old}}(s,a)|
\end{align}$$

#### II.1.6 根据上述条件可以得出保证monotonic improvement的policy iteration算法

- 算法伪码：<font color=brown>详细推导过程见TRPO paper，大体思路和下一节中的目标构建方式相似。</font>
  - 随机初始化策略网路
  - 迭代直到满足收敛条件：
    1. 计算当前策略的advantage$A^{\pi}(s_t,a_t)$
    2. 通过解决下面优化问题更新策略参数：<font color=blue>[下式与paper中L($\bar\theta$)定义不同，因为原文中$\eta(\pi)$不受新策略影响，为书写方便，因此修改]</font>
       - $$\begin{align}
\bar \theta & = \underset{\bar \theta}{argmax}[\bar A(approx) - \frac{4\epsilon \gamma}{(1-\gamma)^2}D_{KL}^{max}(\pi_{\theta}||\pi_{\bar\theta})]\\
\end{align}$$

- <font color=red>该算法无法直接用于实践，因为约束条件太强，step-size过于保守导致迭代速度非常慢。</font>

#### III.4.7 调整为可操作性更强且具备approxmate monotonic improvement特征的目标表达式

- <font color=green>**第一步：将penalty改为有硬约束的优化目标**</font>
$$\begin{align}
& \underset{\color{red}{\bar \theta}}{max} \sum_tE_{s_t\sim P_{\theta}(s_t)}\left[E_{a_t\sim \pi_{\theta }(a_t|s_t)}\frac{\pi_{\color{red}{\bar \theta}}(a_t|s_t)}{\pi_{\theta }(a_t|s_t)} \gamma ^tA^{\pi_{\theta }}(s_t,a_t)\right]\\
& s.t. D_{KL}^{max}(\pi_{\theta }(a_t|s_t)|| \pi_{\bar\theta }(a_t|s_t) ) \le \epsilon 
\end{align}$$

- <font color=green>**第二步：将max KL divergence替换为step-size条件更宽松的一般KL divergence形式**</font>
$$\begin{align}
& \underset{\color{red}{\bar \theta}}{max} \sum_tE_{s_t\sim P_{\theta}(s_t)}\left[E_{a_t\sim \pi_{\theta }(a_t|s_t)}\frac{\pi_{\color{red}{\bar \theta}}(a_t|s_t)}{\pi_{\theta }(a_t|s_t)} \gamma ^tA^{\pi_{\theta }}(s_t,a_t)\right]\\
& s.t. D_{KL}(\pi_{\theta }(a_t|s_t)|| \pi_{\bar\theta }(a_t|s_t) ) \le \epsilon 
\end{align}$$

### II.3 求解优化目标

#### II.3.1 泰勒一阶展开分析上述优化问题

$$\begin{align}
将优化目标简记为：\\
\underset{\bar \theta}{max} f(\bar \theta ) & = \underset{\bar \theta}{max} \sum_tE_{s_t\sim P_{\theta}(s_t)}\left[E_{a_t\sim \pi_{\theta }(a_t|s_t)}[\frac{\pi_{\bar \theta }(a_t|s_t)}{\pi_{\theta }(a_t|s_t)} \gamma ^tA^{\pi_{\theta }}(s_t,a_t)]\right]\\
f(\bar \theta )泰勒一阶展开：\\
f(\bar\theta ) & \approx f(\bar\theta _0) + \nabla_{\bar\theta }f(\bar\theta)|_{\bar\theta=\bar\theta _0}(\bar\theta -\bar\theta _0)\\
取\bar\theta _0=\theta,有：\\
f(\bar\theta ) & \approx f(\theta) + \nabla_{\theta }f(\theta)(\bar\theta -\theta)\\
因为f(\theta )与\bar\theta 无关：\\
\underset{\bar \theta}{max} f(\bar \theta ) & \approx \underset{\bar \theta}{max}\nabla_{\theta }f(\theta)(\bar\theta -\theta)
\end{align}$$

$$\begin{align}
\nabla_{\bar\theta }f(\bar\theta) 
& = \nabla_{\bar\theta }\sum_tE_{s_t\sim P_{\theta}(s_t)}\left[E_{a_t\sim \pi_{\theta }(a_t|s_t)}[\frac{\pi_{\bar \theta}(a_t|s_t)}{\pi_{\theta }(a_t|s_t)} \gamma ^tA^{\pi_{\theta }}(s_t,a_t)]\right]\\
& = \sum_tE_{s_t\sim P_{\theta}(s_t)}\left[E_{a_t\sim \pi_{\theta }(a_t|s_t)}[\frac{\nabla_{\bar\theta }\pi_{\bar \theta}(a_t|s_t)}{\pi_{\theta }(a_t|s_t)} \gamma ^tA^{\pi_{\theta }}(s_t,a_t)]\right]\\
& = \sum_tE_{s_t\sim P_{\theta}(s_t)}\left[E_{a_t\sim \pi_{\theta }(a_t|s_t)}[\frac{\pi_{\bar \theta}(a_t|s_t)}{\pi_{\theta }(a_t|s_t)} \nabla_{\bar\theta }log\pi_{\bar \theta}(a_t|s_t)\gamma ^tA^{\pi_{\theta }}(s_t,a_t)]\right]\\
\end{align}$$

- 取$\bar\theta _0=\theta$时,有：
$$
\begin{align}
f(\theta ) & =f(\bar \theta = \theta ) \\
&   = \sum_tE_{s_t\sim P_{\theta}(s_t)}\left[E_{a_t\sim \pi_{\theta }(a_t|s_t)}[\frac{\pi_{\bar \theta=\theta}(a_t|s_t)}{\pi_{\theta }(a_t|s_t)} \gamma ^tA^{\pi_{\theta }}(s_t,a_t)]\right]\\
& = \sum_tE_{s_t\sim P_{\theta}(s_t)}E_{a_t\sim \pi_{\theta }(a_t|s_t)}[\gamma ^tA^{\pi_{\theta }}(s_t,a_t)]\\
& = E_{\tau\sim P_{\theta}(\tau)}\sum_t[\gamma ^tA^{\pi_{\theta }}(s_t,a_t)]\\
& = J(\theta )
\end{align}
$$

$$\begin{align}
\nabla_{\theta }f(\theta) & = \nabla_{\bar\theta }f(\bar\theta) |_{\bar\theta =\theta } \\
& = \sum_tE_{s_t\sim P_{\theta}(s_t)}\left[E_{a_t\sim \pi_{\theta }(a_t|s_t)}[\frac{\pi_{\bar \theta=\theta}(a_t|s_t)}{\pi_{\theta }(a_t|s_t)} \nabla_{\bar\theta }log\pi_{\bar \theta}(a_t|s_t)|_{\bar \theta=\theta}\gamma ^tA^{\pi_{\theta }}(s_t,a_t)]\right]\\
& = \sum_tE_{s_t\sim P_{\theta}(s_t)}\left[E_{a_t\sim \pi_{\theta }(a_t|s_t)}[ \nabla_{\theta }log\pi_{\theta}(a_t|s_t)\gamma ^tA^{\pi_{\theta }}(s_t,a_t)]\right]\\
& = \nabla_{\theta }J(\theta )
\end{align}$$

- 泰勒一阶展开后，原优化问题转化为：
$$\begin{align}
& \underset{\bar \theta}{max} f(\bar \theta ) \approx \underset{\bar \theta}{max}\nabla_{\theta }f(\theta)^T(\bar\theta -\theta)=\nabla_{\theta }J(\theta )^T(\bar\theta -\theta)\\
& s.t. D_{KL}(\pi_{\bar\theta }(a_t|s_t) || \pi_{\theta }(a_t|s_t)) \le \epsilon 
\end{align}$$

#### II.3.1 Natrual PG使用了同样的优化目标，并用taylor一阶近似和Fisher Information Matrix近似求解该优化问题

- <font color=norange>**思路**：</font>
  1. gradient ascent方法是有约束的优化问题的solution，并且该优化问题中的约束是用L2 norm表达的: $\left\|\bar \theta-\theta\right\|^2 =  (\bar \theta-\theta)^TI(\bar \theta-\theta) \le \epsilon $
  2. KL divergence可以近似表达为Hessian matrix为Fisher-information matrix的矩阵形态：$D_{KL}(\pi_{\bar\theta}||\pi_{\theta}) \approx \frac{1}{2}(\bar \theta-\theta)^TF(\bar \theta-\theta)$
  3. 利用上面两种约束的相似性，可以将KL divergence处理到Gradient Ascend方法的step size中求解目标优化问题

1. Gradient Ascent的learning rate控制step size，本质上是给被迭代的参数加上了distance 限制。

$$\begin{align}
\underset{\bar \theta}{max} L(\bar \theta )&=\nabla_{\theta }J(\theta)^T(\bar \theta-\theta)\\
\nabla_{\bar \theta }L(\bar \theta ) &= \nabla_{\theta }J(\theta)\\
对应的GA迭代式: \\
\bar\theta & = \theta +\alpha \nabla_{\theta }J(\theta) \\
\end{align}$$

2. <font color=blue>**Gradient Ascent是有约束的优化问题的解**：</font>
    - 按照下面方式可以找到对应的有约束优化问题：<font color=brown>[详见附页 ref deepseek]</font>

$$\begin{align}
取\epsilon 与\alpha 的关系式：\\
\alpha &= \sqrt{\frac{\epsilon }{\left \| \nabla_\theta J(\theta ) \right \| ^2}} \nabla_\theta J(\theta )\\
迭代式是下面有约束目标问题的解：\\
\underset{\bar \theta}{max} L(\bar \theta )&=\nabla_{\theta }J(\theta)^T(\bar \theta-\theta)\\
s.t. \ \ \left\|\bar \theta-\theta\right\|^2& \le \epsilon 
\end{align}$$

3. 可以将KL divergence约束转化为类似L2 norm的形态
   - $D_{KL}(\pi_{\bar\theta}||\pi_{\theta}) \approx \frac{1}{2}(\bar \theta-\theta)^TF(\bar \theta-\theta)$  <font color=brown>[证明详见附页 ref deepseek]</font>
   - 其中，$F=E_{\pi_\theta}[\nabla_\theta log \pi_\theta (a|s)\nabla_\theta log \pi_\theta (a|s)^T]$，这个值可以抽样估计

4. 将原目标转化为Gradient ascent solution对应的目标，及其对应的GA solution为：
$$\begin{align}
\underset{\bar \theta}{max}& L(\bar \theta )=\nabla_{\theta }J(\theta)^T(\bar \theta-\theta)\\
s.t. \ \ &\frac{1}{2}(\bar \theta-\theta)^TF(\bar \theta-\theta) \le \epsilon \\
GA\ solution：\\
\theta '& = \theta +\alpha F^{-1}\nabla _\theta J(\theta )\\
\alpha  & = \sqrt{\frac{2\epsilon }{ \nabla_\theta J(\theta )^TF\nabla_\theta J(\theta ) }} 
\end{align}$$

#### II.3.2 TRPO提出了更高效的求解该优化问题的方法：共轭梯度

- 解法略
- 算法伪码：<font color=brown>[ref: spinning up]</font>
  - <img src='pics/trpo.png' width='65%'>

## III. PPO

- 为了将目标转化为可以用一阶工具求梯度的形式，PPO提出了两种算法，PPO和PPO2。分别对应两种优化目标，如下：

### III.1 PPO将硬约束转化为penalty

- 将TRPO中的有约束的优化问题转化为无约束优化问题：
$$\begin{align}
& \underset{\theta }{max} \ \hat E_t \left[ \frac{\pi_{\theta}(a_t|s_t)  }{\pi_{ \theta_{old}}(a_t|s_t)} \hat A_t  \right] \\
& s.t. \ \hat E_t [KL(\pi_{\theta_{old}}(·|s_t) |\pi_{\theta}(·|s_t) )] \le \delta  \\
等价问题：\\
& \underset{\theta }{max} \ \hat E_t \left[ \frac{\pi_{\theta}(a_t|s_t)  }{\pi_{ \theta_{old}}(a_t|s_t)} \hat A_t - \beta KL(\pi_{\theta_{old}}(·|s_t) |\pi_{\theta}(·|s_t) )\right]  
\end{align}$$

- 带来新的问题：
  1. 虽然达到了转为无约束优化问题的目标，现在可以直接使用DL的方法解RL问题。但$\beta$很难取到合适的值。因此在训练过程中要根据KL divergence的实际大小，不断手动调该参数。
  2. 这个形式下，无法达到TRPO迭代中的一个优势：monotonic improvement。

### III.2 PPO2在原目标基础上扩展除了CLIP surrogate loss

#### III.2.1 clipped surrogate loss

$$\begin{align}
r_t(\theta ) & = \frac{\pi_{\theta}(a_t|s_t)  }{\pi_{ \theta_{old}}(a_t|s_t)} \\
L^{CLIP}(\theta ) & = \hat E_t \left[ min[r_t(\theta )\hat A_t, clip(r_t(\theta ), 1-\epsilon, 1+\epsilon )\hat A_t]\right] \\ 
\end{align}$$

- 一种更好理解的写法：
$$\begin{align}
& L^{CLIP}(\theta )  = \hat E_t \left[ min[r_t(\theta )\hat A_t, g(\epsilon ,\hat A_t)]\right] \\
& g(\epsilon ,\hat A_t) = \left\{\begin{matrix}
 (1-\epsilon ) \hat A_t& A\ge 0\\
 (1+\epsilon ) \hat A_t & A\le 0
\end{matrix}\right. , \ \ \ r_t(\theta ) = \frac{\pi_{\theta}(a_t|s_t)  }{\pi_{ \theta_{old}}(a_t|s_t)} \\
\end{align}$$

#### III.2.2 算法

- 算法伪码：<font color=brown>[ref: spinning up]</font>
  - <img src='pics/ppo.png' width='70%'>